In [3]:
import mwclient
import time
import transformers

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Bitcoin']

In [4]:
revs = list(page.revisions())

In [5]:
revs[0]

OrderedDict([('revid', 1229590296),
             ('parentid', 1228816165),
             ('user', 'Vgbyp'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=6, tm_mday=17, tm_hour=17, tm_min=3, tm_sec=36, tm_wday=0, tm_yday=169, tm_isdst=-1)),
             ('comment', 'version update')])

In [6]:
revs = sorted(revs, key=lambda rev: rev["timestamp"]) 

In [7]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [8]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.MessageMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.ScalarMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\SUBRAT\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to forc

In [9]:
edits = {}
for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [10]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [11]:
import pandas as pd
edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [12]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.750000
2009-08-05,1,0.748121,0.000000
2009-08-06,2,0.995746,0.000000
2009-08-14,1,0.930021,0.000000
2009-10-13,2,-0.227500,0.500000
...,...,...,...
2024-05-22,9,-0.167402,0.555556
2024-05-24,1,-0.998527,1.000000
2024-05-26,1,-0.995705,1.000000
2024-06-13,1,0.990891,0.000000


In [15]:
edits_df.index = pd.to_datetime(edits_df.index)
from datetime import datetime
dates = pd.date_range(start="2009-03-08",end=datetime.today())
edits_df = edits_df.reindex(dates, fill_value=0)
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-03-09,0,0.000000,0.00
2009-03-10,0,0.000000,0.00
2009-03-11,0,0.000000,0.00
2009-03-12,0,0.000000,0.00
...,...,...,...
2024-06-15,0,0.000000,0.00
2024-06-16,0,0.000000,0.00
2024-06-17,1,0.942049,0.00
2024-06-18,0,0.000000,0.00


In [16]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()
rolling_edits = rolling_edits.dropna()
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.133333,-0.018351,0.025000
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2024-06-15,0.533333,-0.055453,0.110185
2024-06-16,0.533333,-0.055453,0.110185
2024-06-17,0.566667,-0.024052,0.110185
2024-06-18,0.566667,-0.024052,0.110185


In [17]:
rolling_edits.to_csv("bitcoin_sentiment.csv")